In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

In [2]:
df1=pd.read_csv('F:\mine\website\Phising_Training_Dataset.csv')
df2=pd.read_csv('F:\mine\website\Phising_Testing_Dataset.csv')
df1.shape

(8955, 32)

In [3]:
df1.isnull().sum()

key                            0
having_IP                      0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistica

In [4]:
#sns.countplot(df1.Result)

In [5]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df1, title="website EDA Report" , explorative = True)
# profile.to_notebook_iframe()

In [6]:
# import seaborn as sns
# plt.figure(figsize=(20,10))
# cor = df1.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
# plt.show()

In [7]:
# # we select highly correlated features
# # kal peasons correlation method
def correlation(dataset, threshold):
    col_corr = set()  
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold:
                
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [8]:
corr_features = correlation(df1, 0.8)
len(set(corr_features))

2

In [9]:
df1=df1.drop(corr_features,axis=1)
df1.head()
df2=df2.drop(list(corr_features), axis=1)
df2.head()

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report
0,21338,1,1,1,1,1,-1,1,-1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,-1,0,1
1,21339,1,-1,1,1,-1,0,-1,1,1,1,1,1,-1,-1,-1,1,1,0,1,1,1,-1,-1,0,-1,-1,0,1
2,21340,1,-1,1,1,-1,0,0,-1,-1,-1,1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,1,-1,1,-1,1,1,-1
3,21341,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1
4,21342,1,-1,1,1,-1,1,1,-1,1,1,1,1,0,1,-1,1,1,0,1,1,1,1,1,1,-1,1,0,1


In [10]:
X = df1.drop(['key', 'Result'], axis = 1).reset_index(drop=True)
y=df1[['Result']].reset_index(drop=True)

In [11]:
y

,Result
0,-1
1,-1
2,-1
3,-1
4,1
...,...
8950,1
8951,-1
8952,-1
8953,-1


In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X.loc[train_index], X.loc[test_index]
y_train, y_test = y.loc[train_index], y.loc[test_index]

TRAIN: [5635 7789 7753 ... 6055 1759 5571] TEST: [ 128 4993 1903 ... 1235 5061 6115]


In [13]:
# from sklearn.model_selection import train_test_split
# from sklearn import tree
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=4)

In [14]:
import lightgbm as lgb
 
from lightgbm import LGBMClassifier
model_gb = LGBMClassifier()
model_gb.fit(X_train, y_train)

# Predicting the Target variable
pred = model_gb.predict(X_test)
accuracy = model_gb.score(X_test, y_test)
print(accuracy*100)

C:\Users\harpreet soni\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\harpreet soni\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


96.98492462311557


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gb = GradientBoostingClassifier(n_estimators=200,max_features='sqrt', learning_rate=1,min_samples_split=2,min_samples_leaf=6,
max_depth=20)

In [16]:
clf_gb.fit(X_train, y_train.values.ravel())
clf_gb.score(X_test, y_test)*100

97.15242881072027

In [17]:
clf_gb.score(X_train, y_train)*100

98.85538805136795

In [18]:
predictions_gbc = clf_gb.predict(df2[list(X.columns)])
predictions_gbc

array([ 1, -1, -1, ..., -1,  1,  1], dtype=int64)

In [19]:
submission_gb = pd.DataFrame(predictions_gbc)
df2["Result"] = submission_gb

In [20]:
df2[['key','Result']].to_csv('F:\mine\website\my_submitt_gradient.csv', index=False)

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

neurons =[35]
for activation in ['logistic']:
    for neuron in neurons:
        mlp = MLPClassifier(hidden_layer_sizes=(neuron), activation=activation,
                       solver='lbfgs', 
                    alpha=1e-5,)
        mlp.fit(X_train,y_train.values.ravel())
        nn_predictions = mlp.predict(X_test)
        accuracy_test = accuracy_score(y_test, nn_predictions)
print('training-',mlp.score(X_train,y_train)*100)
print('testing-',mlp.score(X_test,y_test)*100)

training- 98.84142936906755
testing- 95.64489112227805


C:\Users\harpreet soni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [22]:
predictions_mlpnew = mlp.predict(df2[list(X.columns)])
predictions_mlpnew

array([ 1, -1, -1, ..., -1,  1,  1], dtype=int64)

In [23]:
submissionmlpn = pd.DataFrame(predictions_mlpnew)

In [24]:
df2["Result"] = submissionmlpn

In [25]:
df2[['key','Result']].to_csv('F:\mine\website\my_submitt_mlpnew.csv', index=False)

In [26]:
# random forest classifer
from sklearn.ensemble import RandomForestClassifier

In [27]:
clf = RandomForestClassifier(n_estimators =100) 
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

from sklearn import metrics 
print(clf.score(X_train,y_train))
print("ACCURACY OF THE MODEL: ", clf.score(X_test,y_test)*100)

<ipython-input-27-1c32d480c815>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


0.9885538805136795
ACCURACY OF THE MODEL:  97.31993299832496


In [28]:
predictions_r = clf.predict(df2[list(X.columns)])
predictions_r

array([ 1, -1, -1, ..., -1,  1,  1], dtype=int64)

In [29]:
submission4 = pd.DataFrame(predictions_r)

In [30]:
df2["Result"] = submission4

In [31]:
df2[['key','Result']].to_csv('F:\mine\website\my_submitt_decision_random.csv', index=False)

In [32]:
# Gradient boosting Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
xgbmodel = XGBClassifier(learning_rate =0.15,n_estimators=300,
max_depth=10,
min_child_weight=3,
 gamma=0.1,
 colsample_bytree=0.7)
xgbmodel.fit(X_train, y_train)

C:\Users\harpreet soni\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\harpreet soni\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\harpreet soni\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d

[10:47:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              enable_categorical=False, gamma=0.1, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=10,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=2, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [34]:
xgbmodel.fit(X_train,y_train)
xgbmodel.predict(X_test)
print('train',xgbmodel.score(X_train,y_train)*100)
print('testing',xgbmodel.score(X_test,y_test)*100)

[10:47:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
train 98.59017308766053
testing 96.98492462311557


In [35]:
predictions1 = xgbmodel.predict(df2[list(X.columns)])
predictions1

array([ 1, -1, -1, ..., -1,  1,  1], dtype=int64)

In [36]:
submission1 = pd.DataFrame(predictions1)

In [37]:
submission1.head()

,0
0,1
1,-1
2,-1
3,-1
4,1


In [38]:
df2["Result"] = submission1
df2.head()

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,21338,1,1,1,1,1,-1,1,-1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,-1,0,1,1
1,21339,1,-1,1,1,-1,0,-1,1,1,1,1,1,-1,-1,-1,1,1,0,1,1,1,-1,-1,0,-1,-1,0,1,-1
2,21340,1,-1,1,1,-1,0,0,-1,-1,-1,1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,1,-1,1,-1,1,1,-1,-1
3,21341,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,-1
4,21342,1,-1,1,1,-1,1,1,-1,1,1,1,1,0,1,-1,1,1,0,1,1,1,1,1,1,-1,1,0,1,1


In [39]:
df_xg = pd.DataFrame(predictions1)
df_xg.value_counts()

 1    1135
-1     965
dtype: int64

In [40]:
df2[['key','Result']].to_csv('F:\mine\website\my_submitt.csv', index=False)

In [65]:
from catboost import CatBoostClassifier, Pool
model_c = CatBoostClassifier(iterations=30,learning_rate=1)

model_c.fit(X_train, y_train)
preds_class = model_c.predict(X_test)


0:	learn: 0.2350694	total: 174ms	remaining: 5.04s
1:	learn: 0.1687280	total: 183ms	remaining: 2.56s
2:	learn: 0.1452496	total: 191ms	remaining: 1.72s
3:	learn: 0.1287809	total: 200ms	remaining: 1.3s
4:	learn: 0.1161737	total: 208ms	remaining: 1.04s
5:	learn: 0.1107393	total: 216ms	remaining: 865ms
6:	learn: 0.1037460	total: 224ms	remaining: 737ms
7:	learn: 0.0981582	total: 233ms	remaining: 641ms
8:	learn: 0.0931599	total: 242ms	remaining: 566ms
9:	learn: 0.0904206	total: 251ms	remaining: 502ms
10:	learn: 0.0860048	total: 260ms	remaining: 449ms
11:	learn: 0.0828034	total: 269ms	remaining: 403ms
12:	learn: 0.0788635	total: 278ms	remaining: 363ms
13:	learn: 0.0736892	total: 286ms	remaining: 327ms
14:	learn: 0.0688664	total: 295ms	remaining: 295ms
15:	learn: 0.0644786	total: 303ms	remaining: 265ms
16:	learn: 0.0607120	total: 312ms	remaining: 238ms
17:	learn: 0.0583613	total: 320ms	remaining: 214ms
18:	learn: 0.0557953	total: 329ms	remaining: 190ms
19:	learn: 0.0541194	total: 337ms	remainin

In [66]:
from sklearn.metrics import accuracy_score
print("The accuracy is: ",100 *accuracy_score(y_test,preds_class))

The accuracy is:  96.70575097710777


In [67]:
predictions2 = model_c.predict(df2[list(X.columns)])
predictions2

array([ 1, -1, -1, ..., -1,  1,  1], dtype=int64)

In [68]:
submission2 = pd.DataFrame(predictions2)

In [69]:
df2["Result"] = submission2
df2.head()

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,21338,1,1,1,1,1,-1,1,-1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,-1,0,1,1
1,21339,1,-1,1,1,-1,0,-1,1,1,1,1,1,-1,-1,-1,1,1,0,1,1,1,-1,-1,0,-1,-1,0,1,-1
2,21340,1,-1,1,1,-1,0,0,-1,-1,-1,1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,1,-1,1,-1,1,1,-1,-1
3,21341,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,-1
4,21342,1,-1,1,1,-1,1,1,-1,1,1,1,1,0,1,-1,1,1,0,1,1,1,1,1,1,-1,1,0,1,1


In [70]:
df2[['key','Result']].to_csv(r'F:\mine\website\new_submitt_cat.csv', index=False)